# Data Cleaning
***

## Data Aquisition

In [1]:
import pandas as pd
import numpy as np
from numba import njit, jit
from typing import TypeVar
import multiprocessing
from joblib import Parallel, delayed
from IPython.core.display import HTML, display
import time
import datetime
from scipy import stats
import os
import warnings

warnings.filterwarnings('ignore')

pd.set_option("display.max_rows", 8)
pd.set_option("display.max_columns", 20)

num_cores = multiprocessing.cpu_count()
PandasDataFrame = TypeVar('pandas.core.frame.DataFrame')
NaN = np.nan

### Time Series Tables

These are the U.S. and Global time series tables from the CSSE database

In [2]:
# save FIPS data separately
fips = pd.read_csv('../data/population/UID_ISO_FIPS_LookUp_Table.csv')
fips_US = fips[777:] # only include county specific
fips_US = fips_US[fips['Country_Region'] == 'US']
fips_US = fips_US[['FIPS', 'Admin2', 'Province_State', 'Country_Region']]
fips_US = fips_US.rename({'Admin2': 'County'},axis=1)
fips_US['FIPS'] = fips_US['FIPS'].astype('str').str.slice(0,-2)

In [3]:
# Import US cases
cases_US = pd.read_csv("../data/pandemic/time_series_covid19_confirmed_US.csv")
cases_US = cases_US[5:] # exclude US territories

cases_US = cases_US.drop(["FIPS","Combined_Key","code3","iso2", "iso3","UID"], axis=1)
cases_US = cases_US.rename(columns={"Admin2": "County", "Long_": "Long"})

# Import Global Cases
cases_global = pd.read_csv("../data/pandemic/time_series_covid19_confirmed_global.csv")
cases_global = cases_global.rename(columns={"Province/State": "Province_State", 
                                            "Country/Region": "Country_Region"})
cases_global = cases_global.drop(['Lat', 'Long'], axis=1).groupby('Country_Region').sum().reset_index()
cases_global["County"] = NaN

In [4]:
display(HTML("<p style='font-size: 15px'>Global Cases</p>"))
cases_global

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,6/22/2020,6/23/2020,6/24/2020,6/25/2020,6/26/2020,6/27/2020,6/28/2020,6/29/2020,6/30/2020,County
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,29157,29481,29640,30175,30451,30616,30967,31238,31517,NaN
1,Albania,0,0,0,0,0,0,0,0,0,...,1995,2047,2114,2192,2269,2330,2402,2466,2535,NaN
2,Algeria,0,0,0,0,0,0,0,0,0,...,11920,12076,12248,12445,12685,12968,13273,13571,13907,NaN
3,Andorra,0,0,0,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,Western Sahara,0,0,0,0,0,0,0,0,0,...,10,10,10,10,10,10,10,10,10,NaN
185,Yemen,0,0,0,0,0,0,0,0,0,...,967,992,1015,1076,1089,1103,1118,1128,1158,NaN
186,Zambia,0,0,0,0,0,0,0,0,0,...,1430,1477,1489,1497,1531,1531,1557,1568,1594,NaN
187,Zimbabwe,0,0,0,0,0,0,0,0,0,...,512,525,530,551,561,567,567,574,591,NaN


In [5]:
display(HTML("<p style='font-size: 15px'>U.S. Cases</p>"))
cases_US

,County,Province_State,Country_Region,Lat,Long,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,...,6/21/2020,6/22/2020,6/23/2020,6/24/2020,6/25/2020,6/26/2020,6/27/2020,6/28/2020,6/29/2020,6/30/2020
5,Autauga,Alabama,US,32.539527,-86.644082,0,0,0,0,0,...,428,436,447,463,473,482,492,497,521,530
6,Baldwin,Alabama,US,30.727750,-87.722071,0,0,0,0,0,...,415,422,435,449,462,500,539,559,626,663
7,Barbour,Alabama,US,31.868263,-85.387129,0,0,0,0,0,...,271,276,279,287,303,309,314,314,319,322
8,Bibb,Alabama,US,32.996421,-87.125115,0,0,0,0,0,...,124,126,132,138,146,150,158,159,162,167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3257,Southeast Utah,Utah,US,38.996171,-110.701396,0,0,0,0,0,...,31,31,31,32,32,33,34,35,35,36
3258,Southwest Utah,Utah,US,37.854472,-111.441876,0,0,0,0,0,...,1056,1104,1151,1200,1258,1302,1361,1428,1467,1519
3259,TriCounty,Utah,US,40.124915,-109.517442,0,0,0,0,0,...,38,39,39,40,42,45,46,48,48,50
3260,Weber-Morgan,Utah,US,41.271160,-111.914512,0,0,0,0,0,...,666,676,708,734,779,814,846,872,919,954


#### Expand Time Series Data

In [6]:
### Expanding time series ###
# Output is saved to covid_19_time_series_all.csv to prevent rerunning
# Cells should be frozen unless data is updated

In [7]:
cases_total_temp = pd.concat([cases_US, cases_global], sort=False)

In [8]:
def get_rows(row):
    temp = pd.DataFrame(columns=pd.DataFrame(columns=['County','Province_State','Country_Region', 
                                                      'Lat','Long','Date','Total_Cases']))
    for date in row[5:].iteritems():
            new_row = row[:5]
            new_row["Date"] = date[0]
            new_row["Total_Cases"] = date[1]
            temp = pd.concat([temp, new_row.to_frame().transpose()])
    return temp

def convert_time_series():
    cols = cases_total_temp.columns[:5].append(pd.Index(["Date","Total_Cases"]))
    temp = pd.DataFrame(columns=cols)

    row_n = 0
    result = Parallel(n_jobs=num_cores-1)(delayed(get_rows)(j) for i, j in cases_total_temp.iterrows())
    return pd.concat(result)

start_time = time.time()
cases_total = convert_time_series()
end_time = time.time() - start_time
# print("--- %s seconds ---" % (end_time))

In [9]:
# save data in case a kernal restart is required
cases_total['Date'] = pd.to_datetime(cases_total['Date'], cache=True)
cases_total.to_csv("../data/pandemic/covid_19_time_series_all.csv")

In [10]:
# load data if kernal is restarted
cases_total = pd.read_csv("../data/pandemic/covid_19_time_series_all.csv",low_memory=False).iloc[:,1:]
cases_total['Date'] = pd.to_datetime(cases_total['Date'], cache=True)

In [11]:
# Clean US and Global tables

# Pull US cases
cases_US = cases_total[cases_total['Country_Region'] == 'US']

# store US aggregate for later
cases_US_agg = cases_US[cases_US['Province_State'].isnull()]
cases_US_agg = cases_US_agg[['Country_Region', 'Date', 'Total_Cases']]
cases_US = cases_US[cases_US['County'].notnull() & cases_US['Province_State'].notnull()] # remove US aggregate from US table

# Organize table columns
cases_total = cases_total[cases_total['Country_Region'] != 'US'].drop(['Province_State', 'County', 'Lat', 'Long'],axis=1)
cases_total = cases_total.groupby(['Country_Region', 'Date', 'Total_Cases']).sum().reset_index()

**Here I wanted to treat the time series data as one feature rather than separate variabels. I explored several ways to approach this, but with a lack of user friendly solutions, I iteratively expanded each row. I was able to minimize the runtime through multiprocessing.**

In [12]:
display(HTML("<p style='font-size: 15px'>U.S. By Counties</p>"))
cases_US

,County,Province_State,Country_Region,Lat,Long,Date,Total_Cases
0,Autauga,Alabama,US,32.539527,-86.644082,2020-01-22,0
1,Autauga,Alabama,US,32.539527,-86.644082,2020-01-23,0
2,Autauga,Alabama,US,32.539527,-86.644082,2020-01-24,0
3,Autauga,Alabama,US,32.539527,-86.644082,2020-01-25,0
...,...,...,...,...,...,...,...
524212,Weber-Morgan,Utah,US,41.271160,-111.914512,2020-06-27,846
524213,Weber-Morgan,Utah,US,41.271160,-111.914512,2020-06-28,872
524214,Weber-Morgan,Utah,US,41.271160,-111.914512,2020-06-29,919
524215,Weber-Morgan,Utah,US,41.271160,-111.914512,2020-06-30,954


**The global table was filtered by the key countries of interest: U.S., Australia, Canada, China, and UK. I chose these countries as they have province specific data making them comparable at a detailed level. The U.S. cases were kept in a separate table at the county level for separate analysis, but were aggregated to the state level within the global table.**

In [13]:
display(HTML("<p style='font-size: 15px'>Global</p>"))
cases_total

,Country_Region,Date,Total_Cases
0,Afghanistan,2020-01-22,0
1,Afghanistan,2020-01-23,0
2,Afghanistan,2020-01-24,0
3,Afghanistan,2020-01-25,0
...,...,...,...
30103,Zimbabwe,2020-06-27,567
30104,Zimbabwe,2020-06-28,567
30105,Zimbabwe,2020-06-29,574
30106,Zimbabwe,2020-06-30,591


### Adding Temperature, Political Party, Protest, and Lockdown Variables

In [14]:
### COVID-19 Phases ###

In [15]:
# Load lockdown phase data
phases = pd.read_csv("../data/pandemic/WP_phases_6.30.20.csv")
for i in range(4):
    phases[f'Phase.{i}'] = pd.to_datetime(phases[f'Phase.{i}'], cache=True)

cases_US = cases_US.merge(phases, left_on='Province_State', right_on='State')

In [16]:
# determine current phase of each data point
def set_phases(row):
    if  row['Date'] < row['Phase.0']:
        return -1
    elif row['Date'] >= row['Phase.0'] and row['Date'] < row['Phase.1']:
        return 0
    elif row['Date'] >= row['Phase.1'] and row['Date'] < row['Phase.2']:
        return 1
    elif row['Date'] >= row['Phase.2'] and row['Date'] < row['Phase.3']:
        return 2
    else:
        return 3

cases_US['Current_Phase'] = pd.Categorical(cases_US.apply(set_phases, axis=1))

In [17]:
temp1 = pd.DataFrame(columns=['County', 'Province_State', 'Country_Region', 'Lat', 'Long', 'Date',
       'Total_Cases', 'State', 'Abbreviation', 'Phase.0', 'Phase.1', 'Phase.2',
       'Phase.3', 'Restriction Rating', 'Governer.Party', 'Current_Phase',
       'Month', 'Cases_2W'])

start_time = time.time()
for County_State, df in cases_US.groupby(['County', 'State']):
    temp2 = df
    temp2['Cases_2W'] = temp2['Total_Cases'].shift(-14)
    temp1 = pd.concat([temp1,temp2])

end_time = time.time() - start_time
cases_US = temp1.sort_values(['Province_State', 'County', 'Date'], ascending=[True,True,True])

In [18]:
### Temperature Data ###

In [19]:
# data prep for temps
fips_US['County_FIPS'] = fips_US['FIPS'].str.slice(-3)
cases_US = cases_US.merge(fips_US)
cases_US['Month'] = cases_US['Date'].dt.to_period('M')
cases_US['Total_Cases'] = cases_US['Total_Cases'].astype('int')

In [20]:
### US temps
files = []
for r, d, f in os.walk("../data/temperature/US/"):
    for file in f:
        if '.csv' in file:
            files.append(os.path.join(r, file))

dates = pd.date_range('2020-01-31', periods=len(files), freq='M')
temps = pd.DataFrame(columns=pd.DataFrame(columns=["Location ID","Location","Value","Rank",
                                                   "Anomaly (1901-2000 base period)",
                                                   "1901-2000 Mean", "Month"]))
for i, file in enumerate(files):
    temp = pd.read_csv(file)
    temp['Month'] = dates[i]
    temps = pd.concat([temps, temp])
temps['Month'] = temps['Month'].dt.to_period('M')

temps['Abbreviation'] = temps['Location ID'].str.slice(0,2)
temps['County_FIPS'] = temps['Location ID'].str.slice(-3)
temps = temps.drop(['Location ID', 'Location', 'Rank', 'Anomaly (1901-2000 base period)', '1901-2000 Mean'], axis=1)
temps = temps.rename({'Value': 'Avg_Temp'}, axis=1)

temps_US = pd.merge(cases_US, temps, how='left', on=['Abbreviation','Month', 'County_FIPS'])
temps_US = temps_US.drop('Province_State', axis=1)

# fill in missing temps with state average by month
avg_m_temps_us = (temps_US.groupby(['State','Month'])
    .mean()
    .reset_index()
    .drop(['Lat','Long','Total_Cases'], axis=1))
rows = temps_US[np.isnan(temps_US['Avg_Temp'])][['County', 'State', 'Date', 'FIPS', 'Month']]

missing_temps = rows.reset_index().merge(avg_m_temps_us, on=['State', 'Month']).set_index('index')
missing_temps['Avg_Temp'] = round(missing_temps['Avg_Temp'], ndigits=1)

temps_US.loc[missing_temps.index,['Avg_Temp']] = missing_temps['Avg_Temp']
cases_US = temps_US.drop('County_FIPS', axis=1)

In [21]:
# international temps
cases_total['Month'] = cases_total['Date'].dt.to_period('M')

temps_global = pd.read_csv('../data/temperature/UNdata_Export_Global_Temps.csv')
temps_AU = pd.read_csv('../data/temperature/NOAA_Australia_2020_temps.csv')
temps_global = pd.concat([temps_global, temps_AU]).reset_index().iloc[:, 1:]
temps_global = temps_global.iloc[:, :20]
temps_global = temps_global.rename({'Country or Territory': 'Country_Region'}, axis=1)
temps_global.loc[temps_global['Country_Region'] == 'UNITED KINGDOM OF GREAT BRITAIN & NORTHERN IRELAND', 
               'Country_Region'] = 'United Kingdom'



temps_global = temps_global[
    (temps_global['Statistic Description'] == 'Mean Value') & # average temp values
    (np.isnan(temps_global['Jan Footnotes'])) & # NaN means accurate value
    (np.isnan(temps_global['Feb Footnotes'])) &
    (np.isnan(temps_global['Mar Footnotes'])) &
    (np.isnan(temps_global['Apr Footnotes'])) &
    (np.isnan(temps_global['May Footnotes'])) &
    (np.isnan(temps_global['Jun Footnotes']))
]

remove_cols = [i for i in temps_global.columns if 'Footnotes' in i] 
remove_cols.extend(temps_global.iloc[:,1:8].columns)
temps_global = temps_global.drop(list(remove_cols),axis=1)

def get_month(m_str):
    if m_str == 'Jan':
        return '2020-01'
    elif m_str == 'Feb':
        return '2020-02'
    elif m_str == 'Mar':
        return '2020-03'
    elif m_str == 'Apr':
        return '2020-04'
    elif m_str == 'May':
        return '2020-05'
    elif m_str == 'Jun':
        return '2020-06'
    else:
        return ''

def get_rows_temps(row):
    temp = pd.DataFrame(columns=pd.DataFrame(columns=['Country_Region', 'Avg_Temp', 'Month']))
    for month in row[1:].iteritems():
        new_row = pd.DataFrame([[row[0], month[1], get_month(month[0])]],
                              columns=['Country_Region', 'Avg_Temp', 'Month'])
        temp = pd.concat([temp, new_row])
    return temp

def convert_global_temps():
    cols = temps_global.columns[:5].append(pd.Index(["Date","Total_Cases"]))
    temp = pd.DataFrame(columns=cols)

    row_n = 0
    result = Parallel(n_jobs=num_cores-1)(delayed(get_rows_temps)(j) for i, j in temps_global.iterrows())
    return pd.concat(result)

start_time = time.time()
temps_global = convert_global_temps()
end_time = time.time() - start_time

temps_global['Country_Region'] = temps_global['Country_Region'].str.title().str.strip()
temps_global['Month'] = pd.to_datetime(temps_global['Month'].str.strip()).dt.to_period('M')
temps_global = temps_global.groupby(['Country_Region','Month']).mean().reset_index()

temp = cases_total.merge(temps_global, how='left', on=['Country_Region','Month'])
cases_total = temp

In [22]:
### Protest Data ###

In [23]:
BLM_global = pd.read_csv('../data/protests/BLMProtestsCount.csv').iloc[:,1:]
BLM_global = BLM_global.rename({'Value.properties.Country': 'Country_Region',
                                'Region': 'Province_State', 'n': 'Protest_Count',
                                'State': 'Abbreviation'}, axis=1)

# Data Cleaning
BLM_global.loc[BLM_global['Country_Region'] == 'USA', 'Country_Region'] = 'US'

BLM_US = BLM_global[BLM_global['Country_Region'] == 'US']
BLM_US = BLM_US.rename({'Province_State': 'County', 'State': 'Abbreviation'}, axis=1)
BLM_US['County'] = BLM_US['County'].str.replace(' County', '')
BLM_US['County'] = BLM_US['County'].str.replace(' Parish', '')

# US Protests
temp = pd.merge(cases_US, BLM_US, how='left')
temp.loc[temp['Protest_Count'].isnull(), 'Protest_Count'] = 0
temp['Protest_Count'] = temp['Protest_Count'].astype('int32')
cases_US = temp

# Global
# States = cases_US[['State', 'Abbreviation']].groupby(["State", "Abbreviation"]).max().reset_index()
BLM_global = BLM_global.drop(['Province_State', 'Abbreviation'], axis=1)
BLM_global = BLM_global.groupby('Country_Region').sum().reset_index()

cases_total = cases_total.merge(BLM_global, how='left').dropna()

**After organizing the pandemic data, I added a series of other interesting variables mainly to the U.S. dataset including average temperature per month, political party controlling the current local governemnt, a count for the number of recoreded Black Lives Matter protests in the area, COVID-19 lockdown phases and restriction rating from the Washington Post.**

**I added another variable Case_2W that shows the number of cases 2 weeks from the current date (does not go past 6/16/20).**

In [24]:
display(HTML("<p style='font-size: 15px'>U.S. By Counties</p>"))
cases_US

,County,Country_Region,Lat,Long,Date,Total_Cases,State,Abbreviation,Phase.0,Phase.1,Phase.2,Phase.3,Restriction Rating,Governer.Party,Current_Phase,Month,Cases_2W,FIPS,Avg_Temp,Protest_Count
0,Autauga,US,32.539527,-86.644082,2020-01-22,0,Alabama,AL,2020-04-04,2020-04-30,2020-05-11,2020-05-22,minor,R,-1,2020-01,0.0,1001,50.5,0
1,Autauga,US,32.539527,-86.644082,2020-01-23,0,Alabama,AL,2020-04-04,2020-04-30,2020-05-11,2020-05-22,minor,R,-1,2020-01,0.0,1001,50.5,0
2,Autauga,US,32.539527,-86.644082,2020-01-24,0,Alabama,AL,2020-04-04,2020-04-30,2020-05-11,2020-05-22,minor,R,-1,2020-01,0.0,1001,50.5,0
3,Autauga,US,32.539527,-86.644082,2020-01-25,0,Alabama,AL,2020-04-04,2020-04-30,2020-05-11,2020-05-22,minor,R,-1,2020-01,0.0,1001,50.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521636,Weston,US,43.839612,-104.567488,2020-06-27,1,Wyoming,WY,2020-03-15,2020-03-15,2020-05-01,2020-05-15,minor,R,3,2020-06,NaN,56045,66.8,0
521637,Weston,US,43.839612,-104.567488,2020-06-28,1,Wyoming,WY,2020-03-15,2020-03-15,2020-05-01,2020-05-15,minor,R,3,2020-06,NaN,56045,66.8,0
521638,Weston,US,43.839612,-104.567488,2020-06-29,2,Wyoming,WY,2020-03-15,2020-03-15,2020-05-01,2020-05-15,minor,R,3,2020-06,NaN,56045,66.8,0
521639,Weston,US,43.839612,-104.567488,2020-06-30,2,Wyoming,WY,2020-03-15,2020-03-15,2020-05-01,2020-05-15,minor,R,3,2020-06,NaN,56045,66.8,0


**For the global table, I was able to add Average Temp for most non-US countries and an international BLM protest count per country.**

In [25]:
display(HTML("<p style='font-size: 15px'>Global</p>"))
cases_total

,Country_Region,Date,Total_Cases,Month,Avg_Temp,Protest_Count
966,Argentina,2020-01-22,0,2020-01,22.950000,1.0
967,Argentina,2020-01-23,0,2020-01,22.950000,1.0
968,Argentina,2020-01-24,0,2020-01,22.950000,1.0
969,Argentina,2020-01-25,0,2020-01,22.950000,1.0
...,...,...,...,...,...,...
28332,Ukraine,2020-06-27,42932,2020-06,18.389362,1.0
28333,Ukraine,2020-06-28,43856,2020-06,18.389362,1.0
28334,Ukraine,2020-06-29,44538,2020-06,18.389362,1.0
28335,Ukraine,2020-06-30,45254,2020-06,18.389362,1.0


### Population and Demographics Variables (US)

In [26]:
### Popualtion Data ###

In [27]:
# US population
pop_US = pd.read_csv('../data/population/county_area_2010.csv')
pop_US = pop_US[['fips', 'AGE775213', 'RHI125213', 'SEX255213',
                 'RHI225213', 'RHI325213', 'RHI425213', 'RHI525213',
                 'RHI625213', 'RHI725213', 'POP645213', 'HSD310213',
                 'LND110210']]
pop_US = pop_US.rename({'fips': 'FIPS', 'AGE775213': 'Perc.Over.65', 'SEX255213': 'Perc.Female',
                        'RHI125213': 'Perc.White','RHI225213': 'Perc.Black', 
                        'RHI325213': 'Perc.Native', 'RHI425213': 'Perc.Asian',
                        'RHI525213': 'Perc.Pac.Island', 'RHI625213': 'Perc.Mixed',
                        'RHI725213': 'Perc.His.Lat', 'POP645213': 'Perc.Foreign.Born', 
                        'HSD310213': 'Avg.Person.Per.Household',
                        'LND110210': 'Land.Area.sq.mi'},axis=1)
pop_US['FIPS'] = pop_US['FIPS'].astype('str')

# update population to 2019
pop_US_19 = pd.read_csv('../data/population/co-est2019-annres.csv', engine='python')
pop_US_19['POP.2019'] = pop_US_19['2019'].astype('int') / 1000
temp = pop_US_19['Region'].str.split(',',n = 1, expand = True)
pop_US_19['County'] = temp[0].str.replace(' County', '').str.strip()
pop_US_19['Province_State'] = temp[1].str.strip()
pop_US_19 = pop_US_19.drop(['Region','2019'],axis=1)
pop_US_19 = pop_US_19.merge(fips_US)

pop_US = pop_US.merge(pop_US_19)

# update density
pop_US['Area.sq.km'] = (pop_US['Land.Area.sq.mi'] * 2.58999) / 1000
pop_US['PopDensity'] = pop_US['POP.2019'] / pop_US['Area.sq.km']
pop_US = pop_US.drop(['Land.Area.sq.mi'],axis=1)

temp = cases_US.merge(pop_US, how='left')
cases_US = temp[temp['FIPS'].notnull()]

In [28]:
# global population
pop_global = pd.read_csv("../data/population/WPP2019_TotalPopulationBySex.csv")
pop_global = pop_global[pop_global['Time'] == 2020]
pop_global = pop_global.groupby('Location').mean().reset_index()
pop_global = pop_global.drop(['VarID', 'Time', 'MidPeriod'], axis=1)
pop_global = pop_global.rename({'LocID': 'FIPS', 'Location': 'Country_Region'},axis=1) # technically incorrect but makes joining simpler

pop_global['FIPS'] = pop_global['FIPS'].astype('str').str.slice(0,-2)
pop_global['Area.sq.km'] = pop_global['PopTotal'] / pop_global['PopDensity']
pop_global.loc[pop_global['Country_Region'] == 'United States of America', 'Country_Region'] = 'US'
pop_global.loc[pop_global['Country_Region'] == 'United United Kingdom (and dependencies)',
               'Country_Region'] = 'United Kingdom'

# add US aggregate to global
temp1 = pop_global[pop_global['Country_Region'] == 'US']
temp2 = cases_US.groupby(['Country_Region', 'Date']).agg({
    'Protest_Count': np.sum,
    'Avg_Temp': np.mean
}).reset_index()
cases_US_agg = cases_US_agg.merge(temp2).merge(temp1)

cases_total = cases_total.merge(pop_global, how='left')
cases_total = pd.concat([cases_total, cases_US_agg])

**Lastly, I added American Community Survey (ACS) estimates from the Census Bureau including demographic data at the county level in the U.S along with the population density of each county. The population and area metrics are in the thousands.**

In [29]:
display(HTML("<p style='font-size: 15px'>U.S. By Counties</p>"))
pop_US

,FIPS,Perc.Over.65,Perc.White,Perc.Female,Perc.Black,Perc.Native,Perc.Asian,Perc.Pac.Island,Perc.Mixed,Perc.His.Lat,Perc.Foreign.Born,Avg.Person.Per.Household,POP.2019,County,Province_State,Country_Region,County_FIPS,Area.sq.km,PopDensity
0,1001,13.5,78.1,51.5,18.4,0.5,1.1,0.1,1.7,2.7,1.6,2.71,55.869,Autauga,Alabama,US,001,1.539594,36.288146
1,1003,18.1,87.3,51.2,9.5,0.7,0.9,0.1,1.5,4.6,3.6,2.52,223.234,Baldwin,Alabama,US,003,4.117514,54.215719
2,1005,15.9,50.2,46.5,47.6,0.6,0.5,0.2,0.9,4.8,2.9,2.66,24.686,Barbour,Alabama,US,005,2.291830,10.771303
3,1007,14.3,76.3,46.0,22.1,0.4,0.2,0.1,0.9,2.1,1.2,3.03,22.394,Bibb,Alabama,US,007,1.612476,13.887959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,56039,11.8,94.6,48.0,1.5,1.0,1.3,0.1,1.5,15.1,9.1,2.76,23.464,Teton,Wyoming,US,039,10.347994,2.267493
3002,56041,10.5,95.6,49.3,0.8,1.3,0.4,0.2,1.7,9.1,3.4,2.85,20.226,Uinta,Wyoming,US,041,5.390443,3.752197
3003,56043,19.2,95.0,49.7,0.8,1.7,0.7,0.1,1.7,14.2,5.0,2.40,7.805,Washakie,Wyoming,US,043,5.797822,1.346195
3004,56045,17.4,95.3,47.6,0.6,1.7,0.3,0.0,2.1,3.8,0.4,2.29,6.927,Weston,Wyoming,US,045,6.211029,1.115274


**For the global data, the United Nations had population numbers per country. These are 2020 estimates.**

In [30]:
display(HTML("<p style='font-size: 15px'>Global</p>"))
pop_global

,Country_Region,FIPS,PopMale,PopFemale,PopTotal,PopDensity,Area.sq.km
0,Afghanistan,4,19976.265,18952.076,38928.341,59.627,652.864323
1,Africa,903,669878.340,670719.773,1340598.113,45.216,29648.755153
2,African Group,1823,668995.562,669831.029,1338826.591,45.571,29378.916219
3,African Union,1560,669307.822,670116.099,1339423.921,45.182,29645.078151
...,...,...,...,...,...,...,...
470,World Bank Regional Groups (developing only),1801,3299567.992,3229194.235,6528762.227,71.088,91840.567001
471,Yemen,887,15024.985,14800.983,29825.968,56.492,527.967995
472,Zambia,894,9103.006,9280.950,18383.956,24.730,743.386818
473,Zimbabwe,716,7092.010,7770.917,14862.927,38.420,386.853904


In [31]:
temp1 = pd.DataFrame(columns=['County', 'Province_State', 'Country_Region', 'Lat', 'Long', 'Date',
       'Total_Cases', 'State', 'Abbreviation', 'Phase.0', 'Phase.1', 'Phase.2',
       'Phase.3', 'Restriction Rating', 'Governer.Party', 'Current_Phase',
       'Month', 'Cases_2W', 'Cases_Delta'])

for County_State, df in cases_US.groupby(['County', 'State']):
    temp2 = df
    temp2['Cases_Delta'] = temp2['Total_Cases'].shift(-1) - temp2['Total_Cases']
    temp1 = pd.concat([temp1,temp2])
    
cases_US = temp1

In [32]:
cases_US.drop('Province_State', axis=1).to_csv("../data/US_all_vars.csv")
cases_total.to_csv("../data/Global_all_vars.csv")